In [11]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from code_vi.system import OpticsManager
from code_vi.elements import OpticalElement
from code_vi.ray_trace import RayTracer
from code_vi.visualization import Draw


# 1. Initialize
manager = OpticsManager()

lens1 = OpticalElement(
    name="Lens 1", optic_type="Lens",
    x_center=12.7, y_center=51.1, 
    orientation_angle=90.0,         
    clear_aperture=22.86,
    diameter=25.4, 
    center_thickness=3.50,           
    R1=np.inf, R2=-71.260,            
    k2=-1.194332,                    
    coeffs2=[-1.941175e-07, -1.344470e-11], 
    material="ZnSe"
)

lens2 = OpticalElement(
    name="Lens 2", optic_type="Lens",
    x_center=12.7, y_center=156.2,   
    orientation_angle=90.0,          
    clear_aperture=22.86,
    diameter=25.4, 
    center_thickness=3.50,           
    R1=71.260, R2=np.inf,            
    k1=-1.194332,                    
    coeffs1=[-1.941175e-07, -1.344470e-11], 
    material="ZnSe"
)

focal_plane_y = 207.3
focal_plane_offset = 0.0
    
grating1 = OpticalElement(
    name="Grating 1", optic_type="Grating",
    x_center=12.7, y_center=focal_plane_y + focal_plane_offset, 
    orientation_angle=-90,   
    clear_aperture=45, diameter=50.8,
    groove_density=75.0, diffraction_order=-1, material="Gold"
)

grating2 = OpticalElement(
    name="Grating 2", optic_type="Grating",
    x_center=-10, y_center=focal_plane_y - 25 + focal_plane_offset, 
    orientation_angle=-8.5,   
    clear_aperture=45, diameter=50.8,
    groove_density=25.0, diffraction_order=1, material="Gold" # Keep order matching to add dispersion
)

manager.add_element(lens1)
manager.add_element(lens2)
manager.add_element(grating1)
manager.add_element(grating2)


tracer = RayTracer(manager)

# --- SMART GENERATION ---
# Automatically finds the valid grating region and optimizes angles
tracer.generate_smart_spr_source(
    n_sources=11,                # Number of distinct source points you want
    rays_per_source=20,         # Rays per point (uniformly distributed in valid cone)
    target_optic_name="Lens 2", # The optic that defines "success" (can be OAP/Mirror too)
    grating_search_bounds=(0, 25.4), # Max physical length of grating to scan
    acceptance_angle_range=(70, 110), 
    grating_period=10.0,
    beam_energy=0.99
)    

# 3. Run Simulation
print("Running Simulation...")
for t in np.arange(0, 1500, 50.0):
    tracer.run_time_step(t, 50.0)
tracer._sync_to_dataframe()

# 4. VISUALIZATION (Now handled entirely by the class)
# This automatically handles the slider, the figure creation, 
# and preserving your zoom level when switching sources.
Draw.interactive_session(
manager, 
tracer, 
show_curvature=False, 
show_skeleton=True,       # <--- You can toggle these easily now
draw_beam_arrow=True,
show_intersection_points=False
)


--- Smart Source Generation (Target: Lens 2) ---
   Step 1: Finding valid grating length...
   -> Valid Grating Region: 2.67mm to 22.73mm (Extent: 20.05mm)
   Step 2: Optimizing angles for 11 sources...
     Src 1: X=2.7mm | Angles=[89.4°, 91.5°] (20 rays)
     Src 2: X=4.7mm | Angles=[86.5°, 93.7°] (20 rays)
     Src 3: X=6.7mm | Angles=[83.8°, 95.9°] (20 rays)
     Src 4: X=8.7mm | Angles=[81.3°, 98.1°] (20 rays)
     Src 5: X=10.7mm | Angles=[78.9°, 100.2°] (20 rays)
     Src 6: X=12.7mm | Angles=[77.6°, 102.4°] (20 rays)
     Src 7: X=14.7mm | Angles=[79.8°, 100.8°] (20 rays)
     Src 8: X=16.7mm | Angles=[81.9°, 98.6°] (20 rays)
     Src 9: X=18.7mm | Angles=[84.1°, 96.2°] (20 rays)
     Src 10: X=20.7mm | Angles=[86.3°, 93.6°] (20 rays)
     Src 11: X=22.7mm | Angles=[88.5°, 90.6°] (20 rays)
   Done. Generated 220 optimized rays.
Running Simulation...
